In [ ]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Parámetros globales
fs = 2048
duration = 1.0
t = np.linspace(0, duration, int(fs * duration), endpoint=False)

# Función para generar señal y graficar
def generar_senal():
    # Leer amplitudes y frecuencias
    amplitudes = [float(a.get() or 0) for a in amp_entries]
    frecuencias = [float(f.get() or 0) for f in freq_entries]

    # Generar señal
    signal = np.zeros_like(t)
    for A, f in zip(amplitudes, frecuencias):
        signal += A * np.sin(2 * np.pi * f * t)

    # Calcular FFT con ventana Hanning
    N = len(signal)
    window = np.hanning(N)
    signal_windowed = signal * window
    fft_vals = np.fft.fft(signal_windowed)
    freqs = np.fft.fftfreq(N, 1/fs)
    idx = np.where(freqs >= 0)
    freqs = freqs[idx]
    amps = (2 * np.abs(fft_vals[idx])) / N
    amps = amps / np.mean(window)  # Compensación por ventana

    # Calcular RMS
    rms_val = np.sqrt(np.mean(signal**2))
    
    # Calcular THD
    f_nonzero = [f for f in frecuencias if f > 0]
    if f_nonzero:
        f1 = min(f_nonzero)
        harmonics = []
        for n in range(1, 10):
            f_target = n * f1
            idx_h = np.argmin(np.abs(freqs - f_target))
            harmonics.append(amps[idx_h])
        fundamental_amp = harmonics[0]
        other = harmonics[1:]
        thd = np.sqrt(np.sum(np.array(other)**2)) / (fundamental_amp + 1e-12)
    else:
        thd = 0.0

    # Limpiar gráficos anteriores
    for widget in frame_signal.winfo_children():
        widget.destroy()
    for widget in frame_fft.winfo_children():
        widget.destroy()
    for widget in frame_info.winfo_children():
        widget.destroy()

    # Graficar señal
    fig1, ax1 = plt.subplots(figsize=(5, 2))
    ax1.plot(t, signal)
    ax1.set_title("Señal generada")
    ax1.set_xlabel("Tiempo [s]")
    ax1.set_ylabel("Amplitud [V]")
    ax1.grid(True)
    ax1.set_xlim(0, 0.1)  # Zoom en los primeros 0.1 s
    canvas1 = FigureCanvasTkAgg(fig1, master=frame_signal)
    canvas1.draw()
    canvas1.get_tk_widget().pack()

    # Graficar espectro
    fig2, ax2 = plt.subplots(figsize=(5, 2))
    ax2.plot(freqs, amps)
    ax2.set_title("Espectro")
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("Amplitud [V]")
    ax2.set_xlim(0, 500)
    ax2.grid(True)
    canvas2 = FigureCanvasTkAgg(fig2, master=frame_fft)
    canvas2.draw()
    canvas2.get_tk_widget().pack()

    # Mostrar RMS y THD
    tk.Label(frame_info, text=f"RMS: {rms_val:.4f} V").pack()
    tk.Label(frame_info, text=f"THD: {thd*100:.2f}%").pack()

# Crear ventana principal
root = tk.Tk()
root.title("Generador de Señales")

# Entradas para amplitudes y frecuencias
amp_entries = []
freq_entries = []

tk.Label(root, text="AMPLITUD").grid(row=0, column=0, padx=5, pady=5)
tk.Label(root, text="FRECUENCIA").grid(row=1, column=0, padx=5, pady=5)

for i in range(4):  # 4 señales
    a_entry = tk.Entry(root, width=6)
    f_entry = tk.Entry(root, width=6)
    a_entry.grid(row=0, column=i+1, padx=5)
    f_entry.grid(row=1, column=i+1, padx=5)
    amp_entries.append(a_entry)
    freq_entries.append(f_entry)

# Botón GENERAR
btn = tk.Button(root, text="GENERAR", command=generar_senal)
btn.grid(row=0, column=6, rowspan=2, padx=10)

# Frames para gráficos y resultados
frame_signal = tk.Frame(root)
frame_signal.grid(row=2, column=0, columnspan=7, pady=10)

frame_fft = tk.Frame(root)
frame_fft.grid(row=3, column=0, columnspan=7, pady=10)

frame_info = tk.Frame(root)
frame_info.grid(row=4, column=0, columnspan=7, pady=10)

root.mainloop()